In [57]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [69]:
#ruta de las imagenes

healthy_route = 'PlantVillage/train/Grape___healthy'
disease_route = 'PlantVillage/train/Grape___Black_rot'

image = []
tags = []

In [70]:
#funcion para cargar y preprocesar las imagenes

def load_img_from_folder(route, tag):
    for filename in os.listdir(route):
        if filename.endswith('.JPG') or filename.endswith('.png'):
            img = Image.open(os.path.join(route, filename))
            img = img.resize((150, 150))
            img = np.array(img)
            image.append(img)
            tags.append(tag)

In [71]:
#cargar imagenes + convertir las listas en matrices de np

load_img_from_folder(healthy_route, 0)
load_img_from_folder(disease_route, 1)

X = np.array(image)
y = np.array(tags)

In [72]:
#verificar la dimension de los datos 

print("Dimensiones de las imágenes:", X.shape)
print("Dimensiones de las etiquetas:", y.shape)

Dimensiones de las imágenes: (969, 150, 150, 3)
Dimensiones de las etiquetas: (969,)


In [73]:
from sklearn.model_selection import train_test_split

In [74]:
#dividir los datos 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [75]:
#crear un generador de img para el entrenamiento

train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow(
    X_train, y_train,
    batch_size=32
)

In [76]:
#construir la CNN

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid') 
])

In [77]:
#compilar el modelo

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [78]:
#entrenar el modelo

history = model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // 32,
    epochs=10,
    validation_data=(X_test, y_test)
)

Epoch 1/10
24/24 [==============================] - 9s 358ms/step - loss: 0.1097 - accuracy: 0.9529 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/10
24/24 [==============================] - 8s 340ms/step - loss: 0.0217 - accuracy: 0.9838 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
24/24 [==============================] - 9s 348ms/step - loss: 0.0144 - accuracy: 0.9960 - val_loss: 0.0715 - val_accuracy: 0.9948
Epoch 4/10
24/24 [==============================] - 8s 347ms/step - loss: 0.0101 - accuracy: 0.9987 - val_loss: 3.9419 - val_accuracy: 0.9845
Epoch 5/10
24/24 [==============================] - 8s 339ms/step - loss: 0.0083 - accuracy: 0.9973 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/10
24/24 [==============================] - 9s 349ms/step - loss: 0.0064 - accuracy: 0.9987 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/10
24/24 [==============================] - 8s 339ms/step - loss: 0.0103 - accuracy: 0.9973 - val_loss: 0.0000e+00 - val_acc

In [53]:
#Hacer predicciones en una nueva imagen

new_img_path= 'PlantVillage/val/Grape___Black_rot/04bd87b8-f596-4ccc-a9c9-6af129535edc___FAM_B.Rot 0578.JPG'
new_img = Image.open(new_img_path)
new_img = new_img.resize((150,150))
new_img = np.array(new_img) / 255.0
new_img = np.expand_dims(new_img, axis = 0)

In [54]:
prediccion = model.predict(new_img)

if prediccion[0][0] >= 0.5:
    print("La planta está enferma.")
else:
    print("La planta está sana.")

1/1 [==============================] - 0s 18ms/step
La planta está enferma.


In [79]:
#Guardar el modelo

model.save('Modelo_entrenado/modelo_entrenado_maize.h5')